# Install required packages
### We need hl7 library to parse HL7 segments
### Using conda to install helps to install all the dependencies 

In [2]:
# Install libraries 
! pip install -U hl7
%conda install openai==1.12.0 

 
# ! pip install -U hl7
# ! pip install --force-reinstall typing-extensions==4.5
# ! pip install --force-reinstall openai==1.8 --upgrade

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 11, Finished, Available)

Solving environment: - failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /nfs4/pyenv-6199ec3d-8939-46aa-8cde-56150e0244e2

  added / updated specs:
    - openai==1.12.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    annotated-types-0.6.0      |     pyhd8ed1ab_0          17 KB  conda-forge
    anyio-4.3.0                |     pyhd8ed1ab_0         100 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    colorama-0.4.6            

In [82]:
# Import libraries 
import hl7
import pandas as pd
from delta.tables import *




StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 92, Finished, Available)

# Get Openai and Cosmos DB Keys from Keyvault

In [ ]:
from notebookutils.mssparkutils.credentials import getSecret

KEYVAULT_ENDPOINT = "https://<KEYVAULTNAME>.vault.azure.net/"

OPENAI_API_KEY      = getSecret(KEYVAULT_ENDPOINT, "AZURE-OPENAI-KEY")
OPENAI_API_ENDPOINT = getSecret(KEYVAULT_ENDPOINT, "AZURE-OPENAI-KEY")
COSMOS_MONGO_USER   = getSecret(KEYVAULT_ENDPOINT, "COSMOS_MONGO_USER_KEYVAULT")
COSMOS_MONGO_PWD    = getSecret(KEYVAULT_ENDPOINT, "COSMOS_MONGO_PWD_KEYVAULT")
COSMOS_MONGO_SERVER = getSecret(KEYVAULT_ENDPOINT, "COSMOS_MONGO_SERVER_KEYVAULT")

In [83]:
# Path the hl7 file parse
document_path = f"abfss://799a0ea8-6c83-4265-8634-008dccddfe84@onelake.dfs.fabric.microsoft.com/fbcc24f4-768f-460e-88ac-bb668d740476/Files/raw_data/hl7message2.txt"

df = spark.read.format("text").load(document_path)
# .select("_metadata.file_name", "content").limit(10).cache()

# display(df)


StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 93, Finished, Available)

In [84]:
# Collect all rows to a string
hl7_lines = df.collect()

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 94, Finished, Available)

In [85]:
# If you want the HL7 message as a single string variable
hl7_message = "\n".join([row.value for row in hl7_lines])

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 95, Finished, Available)

In [86]:
# Append \r at end of text for proper parsing
hl7_message=hl7_message.replace("\n", "\r") + "\r"

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 96, Finished, Available)

In [87]:
h = hl7.parse(hl7_message)

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 97, Finished, Available)

In [88]:
h['OBX'][0][3][0][1]

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 98, Finished, Available)

['Nota Clínica Completa']

In [89]:
h['OBX'][0][5]

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 99, Finished, Available)

['"El paciente presenta un historial de tres días de dolor abdominal no específico, de intensidad moderada, localizado en el abdomen inferior. No hay síntomas asociados de fiebre, vómitos o diarrea. El historial médico pasado es significativo por episodios similares en el último año, evaluados sin un diagnóstico específico. El examen físico revela una leve sensibilidad en los cuadrantes abdominales inferiores, sin rebote ni defensa. Plan: Se recomienda una ecografía abdominal para descartar cálculos biliares. Puede ser necesaria una evaluación adicional con gastroenterología basada en los resultados de la ecografía."']

In [90]:
h['OBX'][0]

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 100, Finished, Available)

[['OBX'],
 ['1'],
 ['TX'],
 [[['FULLNOTE'], ['Nota Clínica Completa'], ['HospitalSys']]],
 [''],
 ['"El paciente presenta un historial de tres días de dolor abdominal no específico, de intensidad moderada, localizado en el abdomen inferior. No hay síntomas asociados de fiebre, vómitos o diarrea. El historial médico pasado es significativo por episodios similares en el último año, evaluados sin un diagnóstico específico. El examen físico revela una leve sensibilidad en los cuadrantes abdominales inferiores, sin rebote ni defensa. Plan: Se recomienda una ecografía abdominal para descartar cálculos biliares. Puede ser necesaria una evaluación adicional con gastroenterología basada en los resultados de la ecografía."'],
 ['20240301'],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['F']]

# OPEN AI Integration

In [91]:
from openai import AzureOpenAI

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 101, Finished, Available)

In [13]:
azure_client = AzureOpenAI(
  azure_endpoint = OPENAI_API_ENDPOINT, 
  api_key=OPENAI_API_KEY,  
  api_version="2023-09-01-preview"
)

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 23, Finished, Available)

In [110]:
#  Define Methods for Translation and Summarization and create vector embeddings to chat with the data


def translate_to_english_with_gpt4(text):
    response = azure_client.chat.completions.create(
        model="gpt-4",  # Specify GPT-4's engine here
        messages = [
            {"role": "system", "content": "You are an expert in language translation. Your task is to provide insights, advice, and explanations related to language translation, understanding of linguistic nuances, and cultural context in translation. You should not perform direct text translation from one language to another, but rather focus on discussing translation techniques, challenges, and the theory behind language translation."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=1024,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return  response.choices[0].message.content.strip()


def summarize(text):
    response = azure_client.chat.completions.create(
        model="gpt-4",  # Specify GPT-4's engine here
        messages = [
    {"role": "system", "content": "You are an expert in synthesizing medical notes. Your task is to create a brief yet comprehensive summary of the patient's notes. The summary should include key information such as diagnoses, treatment plans, and a clear, ordered list of all medications mentioned. Ensure the summary is concise but contains enough detail to understand the patient's medical situation and treatment regimen."},
    {"role": "user", "content": prompt}
],
        temperature=0.2,
        max_tokens=1024,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return  response.choices[0].message.content.strip()


# Text in an unknown language to be translated
# unknown_language_text = "El paciente presenta un historial de tres días de dolor abdominal no específico, de intensidad moderada, localizado en el abdomen inferior. No hay síntomas asociados de fiebre, vómitos o diarrea. El historial médico pasado es significativo por episodios similares en el último año, evaluados sin un diagnóstico específico. El examen físico revela una leve sensibilidad en los cuadrantes abdominales inferiores, sin rebote ni defensa. Plan: Se recomienda una ecografía abdominal para descartar cálculos biliares. Puede ser necesaria una evaluación adicional con gastroenterología basada en los resultados de la ecografía."
prompt=f"\n\n{h['OBX'][0][5]}"
# Translate the text using GPT-4
english_translation = translate_to_english_with_gpt4(prompt)

# Summarize the notes
summarized = summarize(english_translation)

# Create embeddings on the summarized notes
# embedding_df = azure_client.embeddings.create(model="text-embedding-ada-002",input=summarized)
embedding_df_value = embedding_df.data[0].embedding
print("\nOrignal text:", english_translation)
print("\n\nSummarized text:", summarized)
# print("\n\nSummarized text embeddings: ", embedding_df.data[0].embedding)

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 120, Finished, Available)


Orignal text: The text you provided is a medical report in Spanish. Translating medical documents requires a deep understanding of both the source and target languages, as well as a comprehensive knowledge of the medical field. 

One of the challenges in translating such texts is the use of specific medical terminology. For instance, "cálculos biliares" refers to gallstones, a term that should be accurately translated to avoid any confusion. Similarly, "gastroenterología" is the study of the digestive system and its disorders, which should be translated as "gastroenterology". 

Another challenge is understanding and translating the nuances of the language. For example, "dolor abdominal no específico" literally translates to "non-specific abdominal pain", but in a medical context, it refers to abdominal pain whose cause is not immediately apparent. 

Cultural context is also important in translation. For instance, the phrase "sin rebote ni defensa" literally translates to "without rebo

In [94]:
# Create a Dataframe 
df = pd.DataFrame({
    "Date": "2024",
    "Patient Name": h['PID'][0][4][0][0],
    "DOB": h['PID'][0][6],
    "Original_Notes": h['OBX'][0][5],
    "Translated_Notes": english_translation,
    "Summary": summarized,
    
})

# Display the DataFrame
display(df)

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 104, Finished, Available)

SynapseWidget(Synapse.DataFrame, d9028637-646b-49a0-9519-42ef501dfb75)

In [95]:
# Create embeddings for Summary column
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return azure_client.embeddings.create(input = [text], model=model).data[0].embedding



StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 105, Finished, Available)

In [96]:
# Create a new column with Embeddigs on the Summary text 
df['Vectors'] = df["Summary"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002')) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 106, Finished, Available)

In [97]:
df

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 107, Finished, Available)

,Date,Patient Name,DOB,Original_Notes,Translated_Notes,Summary,Vectors
0,2024,Perez,19750415,"""El paciente presenta un historial de tres día...",Fecha: 1 de marzo de 2024\n\nClinical Note:\n\...,Summary:\n\nThe patient has a three-day histor...,"[-0.0057023386, 0.006620484, 0.044070978, -0.0..."


In [98]:
df.columns

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 108, Finished, Available)

Index(['Date', 'Patient Name', 'DOB', 'Original_Notes', 'Translated_Notes',
       'Summary', 'Vectors'],
      dtype='object')

In [99]:
#  Save to Delta table
# Convert Pandas dataframe to spark dataframe 
# spark_session_results = spark.createDataFrame(df.to_dict)
# df.write.mode("overwrite").format("delta").save("Tables/patient_notes_test")
sparkDF=spark.createDataFrame(df)


StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 109, Finished, Available)

In [74]:
sparkDF.columns

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 84, Finished, Available)

['Date',
 'Patient Name',
 'DOB',
 'Original_Notes',
 'Translated_Notes',
 'Summary',
 'Vectors']

In [100]:
! pip install deltalake

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 110, Finished, Available)

In [101]:
from deltalake import DeltaTable
from deltalake.writer import write_deltalake



StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 111, Finished, Available)

In [102]:
# sparkDF.write.mode("overwrite").format("delta").save("Tables/patient_notes_test")
# Write pandas dataframe to Delta tabel
write_deltalake("Tables/patient_notes_test", df,mode="append")

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 112, Finished, Available)

# Persist Data to Azure CosmosDB MongoDB vCore

In [47]:

! pip install pymongo
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 57, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 90.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is not installed.
dash 2.14.0 requires Flask<2.3.0,>=1.0.4, but you have flask 3.0.0 which is incompatible.
dash 2.14.0 requires Werkzeug<2.3.0, but you have werkzeug 3.0.1 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.10.0 which is incompatible.
     ━━━━━━━━━━━━━

In [103]:
import json
import datetime
import time

import pymongo

from tenacity import retry, wait_random_exponential, stop_after_attempt

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 113, Finished, Available)

In [49]:
COSMOS_MONGO_USER=COSMOS_MONGO_USER_KEYVAULT
COSMOS_MONGO_PWD=COSMOS_MONGO_PWD_KEYVAULT
COSMOS_MONGO_SERVER=COSMOS_MONGO_SERVER_KEYVAULT

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 59, Finished, Available)

In [104]:
mongo_conn = "mongodb+srv://"+COSMOS_MONGO_USER+":"+COSMOS_MONGO_PWD+"@"+COSMOS_MONGO_SERVER+"?tls=true&authMechanism=SCRAM-SHA-256&retrywrites=false&maxIdleTimeMS=120000"
mongo_client = pymongo.MongoClient(mongo_conn)

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 114, Finished, Available)

In [105]:
# create a database called TutorialDB
db = mongo_client['Hl7_raw']

# Create collection if it doesn't exist
COLLECTION_NAME = "patient"

collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 115, Finished, Available)

Using collection: 'patient'.



In [106]:
# Insert into Mongo Collection
data=df.to_dict(orient='records')
collection.insert_many(data)

StatementMeta(, 43561ba3-ad5a-4320-be54-2b6900bfc2f6, 116, Finished, Available)

InsertManyResult([ObjectId('65e5498b0d5b343d8a976110')], acknowledged=True)